<h1>Sandra Field </h1>
MSDS 432, Dr Ali <br>
Spring 2021
<h2>Breadth-First Search in Directed Network Graph</h2><br>

In [48]:
import math
from scipy import stats
import numpy as np
import pandas as pd
import time
import random, string
from time import perf_counter, perf_counter_ns
from random import shuffle, seed
from collections import deque
#from random import seed
#from random import random

#Graphics
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
%matplotlib inline
sns.set_style('white')
#style.use('seaborn-poster')
from IPython.display import HTML



#Network graphing
import networkx
import networkx as nx
from networkx.generators.community import random_partition_graph
from networkx.generators.random_graphs import erdos_renyi_graph
import plotly.graph_objects as go

graph = networkx.Graph()
#G = nx.random_geometric_graph(200, 0.125)
#random_partition_graph


<h1>Create 5-level Directed Connections Graph</h1>
Populated with unique and unduplicated names

In [31]:
np.random.seed(31337)

In [32]:
# Reference networkx graph generators  
# https://networkx.org/documentation/stable/reference/generators.html

In [46]:
#G=random_partition_graph(sizes=(5,5,5,5,5), p_in=1, p_out=0, seed=31337, directed=False)


In [196]:
def create_graph():
    
    #local variables
    levels, children = 5, 5
    graph, ppl_children_lvls = {}, {}
    
    # List the children in each level
    lvl_names = [pow(children, i) for i in range(1, levels + 1)]
    
    # Generate names for all nodes on each level
    for i in lvl_names:
        random.seed(time.time())  #Try to get no duplicates by basing random seed on system time
        ppl_children_lvls[i] = ["".join(random.choices(string.ascii_letters, k=7)) for _ in range(i)]
    
    # Create queues with names from each level
    deqs = [deque(ppl_children_lvls[i]) for i in lvl_names]

    # Initialize names
    for v in ppl_children_lvls.values():
        for name in v:
            graph[name] = []
            
    # Populate children into first 4 levels
    for i, v in enumerate(lvl_names):
        if i == len(lvl_names) - 1:
            break
        for c in ppl_children_lvls[v]:
            if len(graph[c]) == 5:
                continue
            else:
                while len(graph[c]) < 5:
                    graph[c].append(deqs[i + 1].popleft())
    return ppl_children_lvls, graph

#read graph into global var
ppl_children_lvls, graph = create_graph()

In [197]:
# Double check graph size is 5^5 = 3125
cnt, k, v = 0,0,0
for k, v in graph.items():
    if len(v) == 0:
        cnt += 1

print(cnt)

3125


In [198]:
#Check a node and its children
K=2
res = dict(list(graph.items())[0: K])
res
#graph.items()

{'zSHuZVR': ['zSHuZVR', 'XvwjcsQ', 'GoPpJFN', 'GukYpan', 'vezqBGg'],
 'XvwjcsQ': ['QkuDDGG', 'zuCjatd', 'JAPYwgE', 'ZmKmawF', 'BJuSFUP']}

<h1>Breadth-First Search</h1>

In [212]:

def bfs(name: str, lvl: int) -> dict:
    search_queue = deque()
    search_queue += ppl_children_lvls[5]
    # This array is how you keep track of which people you've searched before.
    searched = []
    # Create dict of name and search time.
    prsn = {"name": name, "lvl": lvl}
    start_time = time.perf_counter_ns()
    while search_queue:
        person = search_queue.popleft()
        # Only search this person if you haven't already searched them.
        if person not in searched:
            if person == name:
                end_time = time.perf_counter_ns()
                total_time = (end_time - start_time) / 1000
                prsn["search_time"] = total_time
                return prsn
            else:
                search_queue += graph[person]
                # Marks this person as searched
                searched.append(person)
    return False


In [220]:
# Pick Random Samples from the list of names
def pick_rsamples() -> list:
    search_samples = []
    count = 0
    for k, v in ppl_children_lvls.items(): 
        count += 1
        samples = {}
        samples["level"] = count
        random.seed(time.time())
        samples["names"] = random.choices(v, k=3)
        search_samples.append(samples)
    return search_samples

samples = pick_rsamples()
samples

[{'level': 1, 'names': ['zSHuZVR', 'GoPpJFN', 'GukYpan']},
 {'level': 2, 'names': ['zSHuZVR', 'ThJpVhB', 'Qttxsnh']},
 {'level': 3, 'names': ['GukYpan', 'GzphZAa', 'VqUsyQZ']},
 {'level': 4, 'names': ['qOBRqUV', 'PKEbWzP', 'IDCXqNp']},
 {'level': 5, 'names': ['HTSKMfF', 'ayWQGJX', 'KyreRSJ']}]

<h1>Results</h1>

In [208]:
results = pd.DataFrame(columns = ['Name', 'Level', 'Search Time (ms)'])

In [222]:
def search_test(): 
    search_times = []
    for s in samples:
        for v in s["names"]:
            search_times.append(bfs(v, s["level"]))
    return pd.DataFrame(search_times)


results = search_test()
results

AttributeError: 'bool' object has no attribute 'keys'

<h1>Summary</h1><br>
Although the study is incomplete, a breadth-first search (BFS) requires: <br> 

--Searching entire network which means following each edge (or each arrow, or connection from one node/person to another). So the running time is at least O(number of edges)<br><br>
--Adding one person to the queue takes O(1) time when the person is found, and doing this for each person takes O(number of people). <br><br>
Ultimately, This search searches width before depth, so we expect BFS to take Breadth-first search takes O(number of people) + O(number of edges) or O(V+E), where V=vertices, E=edges.  

<h1> Future studies- Network structure visualization<br><Br>Work in progress</h1>

In [124]:
names = list()    # holds a list of 3200  name charstrings 
N = 7 #each with 7 chars to ensure length

def namegen(str_size=N):
    allowed_chars = string.ascii_letters # options: + string.punctuation + string.digits
    name = list() 
    name = ''.join(random.choices(allowed_chars, k=str_size))
    return name 

##Ensure unique names with randomly generated and compared namelist
def namelist(num_names=3125):  
    for i in range(num_names):
        names.append(namegen())
    return names

names = namelist()  #generate 3125 unsorted names
print(len(names))
names[5]

3125


In [81]:
n=[]
for i in range(1,10,2):
    picknum = ((3200*i)-1)
    n.append(names_unsorted[picknum])
    print("String number ", (picknum+1), " is: ", names_unsorted[picknum])
picknum=3200-1
n.append(names_unsorted[picknum])
print("String number ", (picknum+1), " is: ", names_unsorted[picknum])
#print("\n\nAll strings: \n\n",n)

String number  3200  is:  t


IndexError: list index out of range

In [166]:
import plotly.graph_objects as go
import networkx as nx

#generate graph like above but with  random numbers for testing visual
G = nx.random_geometric_graph(n=3125,radius=5, dim=5)  

In [170]:
len(G)

3125

In [188]:
G.nodes[edge[0]]['pos']  #view nodes/edges at 0

[0.6527274685170925,
 0.41632293010408017,
 0.545669797050604,
 0.5041055398593909,
 0.5564784410700014]

In [189]:
len(G.edges) 

4881250

In [187]:
    a0, b0, c0, d0, e0  = G.nodes[edge[0]]['pos']
    a1, b1, c1, d1, e1  = G.nodes[edge[1]]['pos']
    a2, b2, c2, d2, e2  = G.nodes[edge[2]]['pos']
    a3, b3, c3, d3, e3  = G.nodes[edge[3]]['pos']
    a4, b4, c4, d4, e4  = G.nodes[edge[4]]['pos']

IndexError: tuple index out of range

In [179]:
edge_a, edge_b, edge_c, edge_d, edge_e = [],[],[],[],[]

for edge in G.edges():
    a0, b0, c0, d0, e0  = G.nodes[edge[0]]['pos']
    a1, b1, c1, d1, e1  = G.nodes[edge[1]]['pos']
    a2, b2, c2, d2, e2  = G.nodes[edge[2]]['pos']
    a3, b3, c3, d3, e3  = G.nodes[edge[3]]['pos']
    a4, b4, c4, d4, e4  = G.nodes[edge[4]]['pos']
    edge_a.append(a0)
    edge_a.append(a1)
    edge_a.append(a2)
    edge_a.append(a3)
    edge_a.append(a4)
    edge_a.append(None)
    edge_b.append(b0)
    edge_b.append(b1)
    edge_b.append(b2)
    edge_b.append(b3)
    edge_b.append(b4)
    edge_b.append(None)
    edge_c.append(c0)
    edge_c.append(c1)
    edge_c.append(c2)
    edge_c.append(c3)
    edge_c.append(c4)
    edge_c.append(None)
    edge_d.append(d0)
    edge_d.append(d1)
    edge_d.append(d2)
    edge_d.append(d3)
    edge_d.append(d4)
    edge_d.append(None)
    edge_e.append(e0)
    edge_e.append(e1)
    edge_e.append(e2)
    edge_e.append(e3)
    edge_e.append(e4)
    edge_e.append(None)
edge_trace = go.Scatter(
    a=edge_a, b=edge_b, c=edge_c, d=edge_d, e=edge_e, 
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_a, node_b, node_c, node_d, node_e  = [],[],[],[],[],
for node in G.nodes():
    a, b, c, d, e = G.nodes[node]['pos']
    node_a.append(a)
    node_b.append(b)
    node_c.append(c)
    node_d.append(d)
    node_e.append(e)   
    
    
node_trace = go.Scatter(
    x0=node_a, x1=node_b, x2=node_c, x3=node_d, x4=node_e,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

IndexError: tuple index out of range

In [ ]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [24]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [28]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Directed network graph: Depth=5, Breadth of each node = 5',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

<h3>References</h3><br>
1. Bhargava, A. Y. (2016.) <i>Grokking Algorithms: An Illustrated Guide for Programmers and Other Curious People.</i> Shelter Island, N.Y.: Manning. Chapter 6: Breadth-first search (pages 95–114).<br><br>
2. Graph visualization: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>"
                    